In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  GradientBoostingClassifier
import xgboost as xgb


Проверочная выборка

Для прогноза выбрали XGBoost

In [6]:
df = pd.read_table('C://Users//HP//Desktop//train.tsv', sep='\t')
df = df.drop(["filename"], axis=1)
df_onehot = pd.concat([df.drop('libs', 1), df['libs'].str.get_dummies(sep=",")], 1)
df_virus = df_onehot.loc[df_onehot['is_virus'] == 1].sum()
df_virus= df_virus[1:]
df_virus = pd.Series.to_frame(df_virus)
df_virus= df_virus.set_axis(['lib'], axis=1)
x= df_onehot.drop(["is_virus"], axis = 1)
y = df_onehot["is_virus"]
x

C:\Users\HP\AppData\Local\Temp\ipykernel_17020\571117239.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_onehot = pd.concat([df.drop('libs', 1), df['libs'].str.get_dummies(sep=",")], 1)
C:\Users\HP\AppData\Local\Temp\ipykernel_17020\571117239.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_onehot = pd.concat([df.drop('libs', 1), df['libs'].str.get_dummies(sep=",")], 1)


,.rsrc,.text1,3cxuicontrols.dll,3cxvoipphone.dll,ab.dll,abview.dll,acad.exe,acadaptersinfo.dll,acdb17.dll,ace.dll,...,xolehlp.dll,xpcom.dll,xpsservices.dll,xtp9601lib.dll,zlib.dll,zlib1.dll,zlibwapi.dll,zloadlibrarya,znzin.dll,zsikdy.dll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16288,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_val = pd.read_table('C://Users//HP//Desktop//val.tsv', sep='\t')
df_val = pd.concat([df_val.drop('libs', 1), df_val['libs'].str.get_dummies(sep=",")], 1)
y_test = df_val["is_virus"]
df_val = df_val.drop(["is_virus", "filename"], axis = 1)

C:\Users\HP\AppData\Local\Temp\ipykernel_17020\406616936.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_val = pd.concat([df_val.drop('libs', 1), df_val['libs'].str.get_dummies(sep=",")], 1)
C:\Users\HP\AppData\Local\Temp\ipykernel_17020\406616936.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_val = pd.concat([df_val.drop('libs', 1), df_val['libs'].str.get_dummies(sep=",")], 1)


In [3]:
df_test = pd.read_table('C://Users//HP//Desktop//test.tsv', sep='\t')
df_test = pd.concat([df_test.drop('libs', 1), df_test['libs'].str.get_dummies(sep=",")], 1)
df_test

C:\Users\HP\AppData\Local\Temp\ipykernel_17020\2841979640.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_test = pd.concat([df_test.drop('libs', 1), df_test['libs'].str.get_dummies(sep=",")], 1)
C:\Users\HP\AppData\Local\Temp\ipykernel_17020\2841979640.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_test = pd.concat([df_test.drop('libs', 1), df_test['libs'].str.get_dummies(sep=",")], 1)


,ace.dll,addressbook-vc140-mt-32.dll,advapi32.dll,apdadrv.dll,api-ms-win-appmodel-runtime-l1-1-0.dll,api-ms-win-core-apiquery-l1-1-0.dll,api-ms-win-core-atoms-l1-1-0.dll,api-ms-win-core-com-l1-1-0.dll,api-ms-win-core-com-l1-1-1.dll,api-ms-win-core-console-l1-1-0.dll,...,wldap32.dll,ws2_32.dll,wsnmp32.dll,wsock32.dll,wtsapi32.dll,xerces-vc140-mt-32-3_2_1.dll,xinput1_4.dll,xmllite.dll,zf_cef.dll,zlib1.dll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1196,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1198,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
col_list = list(set().union(x.columns, df_test.columns))
df_test = df_test.reindex(columns=col_list, fill_value=0)
common_cols = list(set(df_test.columns).intersection(x.columns))
df_test = df_test[common_cols]
df_test =df_test.reindex(sorted(df_test.columns), axis=1)
df_test

,.rsrc,.text1,3cxuicontrols.dll,3cxvoipphone.dll,ab.dll,abview.dll,acad.exe,acadaptersinfo.dll,acdb17.dll,ace.dll,...,xolehlp.dll,xpcom.dll,xpsservices.dll,xtp9601lib.dll,zlib.dll,zlib1.dll,zlibwapi.dll,zloadlibrarya,znzin.dll,zsikdy.dll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Байесовская оптимизация по гиперпараметрам

In [14]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from xgboost import XGBClassifier

xgb = XGBClassifier()

search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}

opt = BayesSearchCV(xgb, search_space, cv=3, n_iter=10, scoring='roc_auc', random_state=8) 

numpy.int (наряду с другими подобными типами) устарел

In [17]:
np.int = int
opt.fit(x, y)

C:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:50:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "clf__colsample_bylevel", "clf__colsample_bynode", "clf__colsample_bytree", "clf__gamma", "clf__learning_rate", "clf__max_depth", "clf__reg_alpha", "clf__reg_lambda", "clf__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:50:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "clf__colsample_bylevel", "clf__colsample_bynode", "clf__colsample_bytree", "clf__gamma", "clf__learning_rate", "clf__max_depth", "clf__reg_alpha", "clf__reg_lambda", "clf__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\HP\anaconda3\Lib\site-packages\

C:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:51:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "clf__colsample_bylevel", "clf__colsample_bynode", "clf__colsample_bytree", "clf__gamma", "clf__learning_rate", "clf__max_depth", "clf__reg_alpha", "clf__reg_lambda", "clf__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:51:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "clf__colsample_bylevel", "clf__colsample_bynode", "clf__colsample_bytree", "clf__gamma", "clf__learning_rate", "clf__max_depth", "clf__reg_alpha", "clf__reg_lambda", "clf__subsample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\HP\anaconda3\Lib\site-packages\

BayesSearchCV(cv=3,
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=None...
                             'clf__learning_rate': Real(low=0.001, high=1.0, prior='log-uniform', transform='normalize'),
                             'clf__max_depth': Integer(low=2, high=8, prior='uniform', transform='normalize'),
                             'clf__reg_alpha': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__reg_lambda': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')})

In [60]:
otvet = opt.predict(df_test)
otvet

array([1, 0, 0, ..., 1, 1, 1])

In [22]:
otvet.sum()

898

In [23]:
y.sum()

10860

In [43]:
y_test.sum()

800

Для проверочной выборки требуется создать файл prediction.txt, в котором для каждой строки файла проверочной выборки будет содержаться один символ: либо 1 если модель предсказывает этот файл как зловредный, либо 0 иначе.

In [65]:
np.savetxt('C:/Users/HP/Desktop/prediction.txt', otvet, fmt='%d', delimiter=" ")